In [2]:
import os
import re
import pandas as pd

In [2]:
import os
import re
import pandas as pd

folder_path = '../Simulation/Assets/Results/Correlation_between_obj_and_Cmax'

column_name = ['File_name', 'Type',  'C_max', 'C_avg']
values = []

# get folder
for folder_name in os.listdir(folder_path):
    if not folder_name.endswith('meta'):
        cost_type = folder_name.split('_')[-1]
        if not cost_type[0].isupper():
            cost_type = cost_type[0].upper() + cost_type[1:]

        # get files in folder
        folder_full_path = os.path.join(folder_path, folder_name)

        # Iterate over files in the folder
        for file_name in os.listdir(folder_full_path):
            if not file_name.endswith('meta'):
                # 정규 표현식을 사용하여 'prev'와 'now'를 기준으로 파일 이름 분리
                match = re.match(r'result_(prev.*?)_with_(now.*?)\.csv', file_name)
                if match:
                    prev_part  = match.group(1)
                    now_part = match.group(2) + '.csv'

                    # "RoutePoints"를 "Truck"으로 바꿈
                    prev_file_name = prev_part.replace("RoutePoints", "Truck")
                    now_file_name = now_part.replace("RoutePoints", "Truck")

                    # read csv file
                    file_full_path = os.path.join(folder_full_path, file_name)
                    df = pd.read_csv(file_full_path)
                    # print(df.columns)
                    c_max_prev = df['C_max_prev'][0]
                    c_max_now = df['C_max_now'][0]

                    c_avg_prev = df.loc[df['Truck_id'].str[6:].astype(int)<100, 'Completion_Time'].mean()
                    c_avg_now = df.loc[df['Truck_id'].str[6:].astype(int)>=100, 'Completion_Time'].mean()

                    # Append values to the list
                    prev_value = [prev_file_name, cost_type, c_max_prev, c_avg_prev]
                    values.append(prev_value)
                    now_value = [now_file_name, cost_type, c_max_now, c_avg_now]
                    values.append(now_value)

# create dataframe
cost_type_df = pd.DataFrame(values, columns=column_name)
print(cost_type_df)


                               File_name Type    C_max     C_avg
0    prev_Truck_20_LP_40_50_10_10rep.csv  Max  544.080  438.8640
1     now_Truck_20_LP_40_50_10_10rep.csv  Max  500.872  445.4996
2    prev_Truck_20_LP_40_50_10_11rep.csv  Max  488.016  432.9184
3     now_Truck_20_LP_40_50_10_11rep.csv  Max  489.456  429.3140
4    prev_Truck_20_LP_40_50_10_12rep.csv  Max  492.272  440.7772
..                                   ...  ...      ...       ...
115    now_Truck_20_LP_40_50_10_7rep.csv  Min  422.712  377.1476
116   prev_Truck_20_LP_40_50_10_8rep.csv  Min  425.624  385.2244
117    now_Truck_20_LP_40_50_10_8rep.csv  Min  409.768  367.6772
118   prev_Truck_20_LP_40_50_10_9rep.csv  Min  404.680  377.3072
119    now_Truck_20_LP_40_50_10_9rep.csv  Min  450.768  373.1424

[120 rows x 4 columns]


In [4]:
cost_type_df[cost_type_df['File_name'] == 'prev_Truck_20_LP_40_50_10_1rep.csv']

,File_name,Type,C_max,C_avg
20,prev_Truck_20_LP_40_50_10_1rep.csv,Max,498.736,431.6212
60,prev_Truck_20_LP_40_50_10_1rep.csv,Middle,480.920,416.0424
100,prev_Truck_20_LP_40_50_10_1rep.csv,Min,420.216,388.5020


In [6]:
original_file_name = 'C:\\Users\\USER\\workspace\\TSB\\Model\\correlation_between_obj_and_Cmax.csv'

original_df = pd.read_csv(original_file_name)


# type열 공백제거
original_df['Type'] = original_df['Type'].str.strip()
print(original_df)

                               File_name    Type  Objective_value
0     prev_Truck_20_LP_40_50_10_1rep.csv     Min            25570
1      now_Truck_20_LP_40_50_10_1rep.csv     Min            22648
2     prev_Truck_20_LP_40_50_10_1rep.csv  Middle            37926
3      now_Truck_20_LP_40_50_10_1rep.csv  Middle            37414
4     prev_Truck_20_LP_40_50_10_1rep.csv     Max            44443
..                                   ...     ...              ...
115   now_Truck_20_LP_40_50_10_20rep.csv     Min            19611
116  prev_Truck_20_LP_40_50_10_20rep.csv  Middle            37708
117   now_Truck_20_LP_40_50_10_20rep.csv  Middle            33296
118  prev_Truck_20_LP_40_50_10_20rep.csv     Max            43890
119   now_Truck_20_LP_40_50_10_20rep.csv     Max            41181

[120 rows x 3 columns]


In [7]:
original_df[original_df['File_name'] == 'prev_Truck_20_LP_40_50_10_1rep.csv']

,File_name,Type,Objective_value
0,prev_Truck_20_LP_40_50_10_1rep.csv,Min,25570
2,prev_Truck_20_LP_40_50_10_1rep.csv,Middle,37926
4,prev_Truck_20_LP_40_50_10_1rep.csv,Max,44443


In [8]:
# 'File_name'과 'Type'이 같은 행끼리 병합
merged_df = pd.merge(original_df, cost_type_df, on=['File_name', 'Type'], how='inner')
print(merged_df)

                               File_name    Type  Objective_value    C_max  \
0     prev_Truck_20_LP_40_50_10_1rep.csv     Min            25570  420.216   
1      now_Truck_20_LP_40_50_10_1rep.csv     Min            22648  434.192   
2     prev_Truck_20_LP_40_50_10_1rep.csv  Middle            37926  480.920   
3      now_Truck_20_LP_40_50_10_1rep.csv  Middle            37414  477.056   
4     prev_Truck_20_LP_40_50_10_1rep.csv     Max            44443  498.736   
..                                   ...     ...              ...      ...   
115   now_Truck_20_LP_40_50_10_20rep.csv     Min            19611  420.544   
116  prev_Truck_20_LP_40_50_10_20rep.csv  Middle            37708  455.968   
117   now_Truck_20_LP_40_50_10_20rep.csv  Middle            33296  461.824   
118  prev_Truck_20_LP_40_50_10_20rep.csv     Max            43890  510.808   
119   now_Truck_20_LP_40_50_10_20rep.csv     Max            41181  507.488   

        C_avg  
0    388.5020  
1    380.9956  
2    416.0424  

In [11]:
merged_df[merged_df['File_name'] == 'prev_Truck_20_LP_40_50_10_10rep.csv']

,File_name,Type,Objective_value,C_max,C_avg
54,prev_Truck_20_LP_40_50_10_10rep.csv,Min,20872,415.888,368.5212
56,prev_Truck_20_LP_40_50_10_10rep.csv,Middle,39732,464.504,416.8088
58,prev_Truck_20_LP_40_50_10_10rep.csv,Max,47559,544.080,438.8640


In [12]:
# dataframe to csv
merged_df.to_csv('correlation_between_obj_and_Cmax_final.csv', index=False)